<a href="https://colab.research.google.com/github/rddiasbk/Deliveries_Food_Goods/blob/main/Delivery_Food_and_Goods.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <font color=darkblue size=05> Limpeza e formatação do dataset Delivery Center: Food & Goods orders in Brazil </font>
disponível no [Kaggle](https://www.kaggle.com/datasets/nosbielcs/brazilian-delivery-center/)


In [1]:
#!pip install -- upgrade gspread

In [2]:
import pandas as pd
import numpy as np

# Autorizando acesso necessário para acessar tabelas Google Sheet.

In [3]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [4]:
spreadsheet_channels = gc.open('channels')
spreadsheet_deliveries = gc.open('deliveries')
spreadsheet_drivers = gc.open('drivers')
spreadsheet_hubs = gc.open('hubs')
spreadsheet_payments = gc.open('payments')
spreadsheet_stores = gc.open('stores')

# Definindo as abas do arquivo aberto.

In [5]:
worksheet_channels = spreadsheet_channels.sheet1
worksheet_deliveries = spreadsheet_deliveries.sheet1
worksheet_drivers = spreadsheet_drivers.sheet1
worksheet_hubs = spreadsheet_hubs.sheet1
worksheet_payments = spreadsheet_payments.sheet1
worksheet_stores = spreadsheet_stores.sheet1

In [6]:
df_channels = pd.DataFrame(worksheet_channels.get_all_records())
df_deliveries = pd.DataFrame(worksheet_deliveries.get_all_records())
df_drivers = pd.DataFrame(worksheet_drivers.get_all_records())
df_hubs = pd.DataFrame(worksheet_hubs.get_all_records())
df_payments = pd.DataFrame(worksheet_payments.get_all_records())
df_stores = pd.DataFrame(worksheet_stores.get_all_records())

In [84]:
df_orders = pd.read_csv('/content/drive/MyDrive/Dados-Datas/Delivery/orders.csv')

# Compilando todas as tabelas.

In [85]:
df1 = pd.merge(df_orders, df_payments, how = 'right', on = 'payment_order_id')
places = pd.merge(df_hubs, df_stores, how = 'left', on = 'hub_id')#12 colunas
df2 = pd.merge(places, df1, left_on = 'store_id', right_on = 'store_id') # 40 colunas (-2 id = ok)
drivers_deliveries = pd.merge(df_drivers, df_deliveries, how = 'left', on = 'driver_id')
df3 = pd.merge(drivers_deliveries, df2,how = 'left',on = 'delivery_order_id')

In [102]:
df = pd.merge(df3, df_channels,how = 'left', on ='channel_id')
#CERTO - 53 LINHAS

# A tabela compilada conta com 414.668 linhas e 53 colunas.

In [103]:
df.shape #(414668, 53)

(414668, 53)

##Checando dados nulos

##1. Verificar qual a quantidade dados NaN


In [ ]:
df.isnull().sum()

**Observação:** Investiguei a presença de dados duplicados (`df.col.nunique())`entretanto optei mantê-los pois trazem informações complementares. 
Por exemplo, eliminar dados duplicados utilizando `order_id` afetaria informações de pagamento, pois há pagamentos diferentes (vouchers) para o mesmo pedido.

In [ ]:
null_cols = [x for x in df.columns if df[x].isnull().sum() >= 1]
for x in null_cols:
    print(f"{x}:", "contém", np.round(df[x].isnull().mean() * 100, 2),
          '% dos dados nulos', 'e um total de',
          np.round(df[x].isnull().sum(), 4,), "linhas afetadas")

Excluindo as colunas relacionadas a data do pedido e mantendo apenas coluna de criação do pedido.

In [106]:
colunas = ['order_created_hour','order_created_minute',
            'order_created_day','order_created_month',
            'order_created_year']
           
df.drop(columns = colunas, axis = 1, inplace = True)

### Removendo a coluna `order_moment_delivered`
* Pois os pedidos são mais finalizados do que assinalados como entregues, embora seja uma informação que traria mais uma possibilidade de análise quanto a diferença entre produtos entregues e vendas finalizadas, há mais de 90% de dados vazios, portanto justificando a exclusão.

In [107]:
df.drop(columns = 'order_moment_delivered', inplace = True)

##Preenchendo os valores nulos da `order_amount` com a média dos valores da própria coluna.

In [108]:
df['order_amount'].fillna(df['order_amount'].mean(), inplace = True)

##Preenchendo os valores nulos da `order_delivery_fee` com a média dos valores da própria coluna.

In [109]:
df['order_delivery_fee'].fillna(df['order_delivery_fee'].mean(), inplace = True)

##Atribuindo o valor "Não Informado" a coluna `order_status`




In [110]:
df['order_status'].fillna('NÃO INFORMADO', inplace = True)


###Atribuindo valores “não informados” a coluna `delivery_status`.

In [111]:
df['delivery_status'].fillna('NÃO INFORMADO', inplace = True)

# Alterando colunas do pedido para o tipo data

In [121]:
colunas_datas = ['order_moment_created',
'order_moment_accepted',
'order_moment_ready',
'order_moment_collected',
'order_moment_in_expedition',
'order_moment_delivering',
'order_moment_finished']

for colunas in colunas_datas:
    df[colunas] = df[colunas].astype('datetime64[ns]')

In [125]:
df.columns

Index(['driver_id', 'driver_modal', 'driver_type', 'delivery_id',
       'delivery_order_id', 'delivery_distance_meters', 'delivery_status',
       'hub_id', 'hub_name', 'hub_city', 'hub_state', 'hub_latitude',
       'hub_longitude', 'store_id', 'store_name', 'store_segment',
       'store_plan_price', 'store_latitude', 'store_longitude', 'order_id',
       'channel_id', 'payment_order_id', 'order_status', 'order_amount',
       'order_delivery_fee', 'order_delivery_cost', 'order_moment_created',
       'order_moment_accepted', 'order_moment_ready', 'order_moment_collected',
       'order_moment_in_expedition', 'order_moment_delivering',
       'order_moment_finished', 'order_metric_collected_time',
       'order_metric_paused_time', 'order_metric_production_time',
       'order_metric_walking_time', 'order_metric_expediton_speed_time',
       'order_metric_transit_time', 'order_metric_cycle_time', 'payment_id',
       'payment_amount', 'payment_fee', 'payment_method', 'payment_status

In [124]:
#df.to_csv('deliveries_cleaned')